In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import dask.array as da
import statsmodels.api as sm
from functools import partial
import dask.array as da
import dask.delayed as dd

In [2]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), 'Documents', 'PhD'))
import constants
sys.path.append(constants.MODULE_DIR)

import stats

In [3]:
# client.close()

In [4]:
from dask.distributed import Client
client = Client()
client

- State start
-   Scheduler at:     tcp://127.0.0.1:33735
-   dashboard at:         /proxy/8787/status
- Registering Worker plugin shuffle
-         Start Nanny at: 'tcp://127.0.0.1:42323'
-         Start Nanny at: 'tcp://127.0.0.1:44195'
-         Start Nanny at: 'tcp://127.0.0.1:42121'
-         Start Nanny at: 'tcp://127.0.0.1:41255'
-         Start Nanny at: 'tcp://127.0.0.1:37189'
-         Start Nanny at: 'tcp://127.0.0.1:38547'
-         Start Nanny at: 'tcp://127.0.0.1:44639'
- Register worker <WorkerState 'tcp://127.0.0.1:39089', name: 6, status: init, memory: 0, processing: 0>
- Starting worker compute stream, tcp://127.0.0.1:39089
- Starting established connection to tcp://127.0.0.1:59628
- Register worker <WorkerState 'tcp://127.0.0.1:39049', name: 0, status: init, memory: 0, processing: 0>
- Starting worker compute stream, tcp://127.0.0.1:39049
- Starting established connection to tcp://127.0.0.1:59606
- Register worker <WorkerState 'tcp://127.0.0.1:46239', name: 4, status

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 251.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33735,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.18 GiB
Comm: tcp://127.0.0.1:39049,Total threads: 4
Dashboard: /proxy/38415/status,Memory: 35.88 GiB
Nanny: tcp://127.0.0.1:42323,


In [5]:
# Create sample data
latitudes = np.linspace(-90, 90, 1000)  # Sample latitudes
longitudes = np.linspace(-180, 180, 100)  # Sample longitudes
times = pd.date_range("2024-01-01", periods=500, freq="D")  # 100 time steps

# Create random data for the DataArray
data = np.random.rand(len(times), len(latitudes), len(longitudes))

# Create the xarray DataArray
data_xarray = xr.DataArray(
    data,
    dims=("time", "latitude", "longitude"),
    coords={
        "time": times,
        "latitude": latitudes,
        "longitude": longitudes
    }
).chunk({'time':-1, 'latitude':250})

data_xarray.name = 'tas'

In [6]:
data_xarray

<xarray.DataArray 'tas' (time: 500, latitude: 1000, longitude: 100)>
dask.array<xarray-<this-array>, shape=(500, 1000, 100), dtype=float64, chunksize=(500, 250, 100), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2025-05-14
  * latitude   (latitude) float64 -90.0 -89.82 -89.64 ... 89.64 89.82 90.0
  * longitude  (longitude) float64 -180.0 -176.4 -172.7 ... 172.7 176.4 180.0

In [18]:
time_axis_num = data_xarray.get_axis_num('time')
time_axis_num

0

In [137]:
data_xarray_stacked = data_xarray.stack(stacked_coord=['latitude', 'longitude'])

In [138]:
data_darray = data_xarray_stacked.data.compute()

In [38]:
data_darray

array([[0.23615113, 0.39523368, 0.10132213, ..., 0.09294841, 0.7612314 ,
        0.88686265],
       [0.20218727, 0.23250198, 0.11275673, ..., 0.22445169, 0.42767779,
        0.60122851],
       [0.03197152, 0.86891743, 0.55929358, ..., 0.40957749, 0.87562772,
        0.74729744],
       ...,
       [0.0796718 , 0.17112962, 0.10080427, ..., 0.31994832, 0.40981205,
        0.35895969],
       [0.62279694, 0.19663079, 0.29621088, ..., 0.36222975, 0.66343578,
        0.58946512],
       [0.97280467, 0.8348507 , 0.8077798 , ..., 0.53366608, 0.15899196,
        0.753896  ]])

In [50]:
data_darray.shape

(500, 100000)

In [51]:
data_darray[0].shape

(100000,)

In [42]:
def my_mean(arr):
    return np.mean(arr)

In [57]:
%%timeit
futures = []
for arr in data_darray:
    future = client.submit(my_mean, arr)
    futures.append(future)

results = client.gather(futures)

199 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
output = da.apply_along_axis(my_mean, 0, data_darray)

In [59]:
%%timeit
np.apply_along_axis(my_mean, 0, data_darray)

1.11 s ± 33.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Testing with polfit

In [67]:
stats.polynomial_fit

<function stats.polynomial_fit(y: Union[numpy._typing._array_like._SupportsArray[numpy.dtype], numpy._typing._nested_sequence._NestedSequence[numpy._typing._array_like._SupportsArray[numpy.dtype]], bool, int, float, complex, str, bytes, numpy._typing._nested_sequence._NestedSequence[Union[bool, int, float, complex, str, bytes]]], x: Union[numpy._typing._array_like._SupportsArray[numpy.dtype], numpy._typing._nested_sequence._NestedSequence[numpy._typing._array_like._SupportsArray[numpy.dtype]], bool, int, float, complex, str, bytes, numpy._typing._nested_sequence._NestedSequence[Union[bool, int, float, complex, str, bytes]], NoneType] = None, order: float = None, deg: float = None) -> Union[numpy._typing._array_like._SupportsArray[numpy.dtype], numpy._typing._nested_sequence._NestedSequence[numpy._typing._array_like._SupportsArray[numpy.dtype]], bool, int, float, complex, str, bytes, numpy._typing._nested_sequence._NestedSequence[Union[bool, int, float, complex, str, bytes]]]>

In [70]:
x = np.arange(data_darray.shape[0])
x[:3]

array([0, 1, 2])

In [84]:
polyfit_2 = partial(stats.polynomial_fit, order=2)

In [85]:
data_darray.shape

(500, 100000)

In [86]:
from importlib import reload

In [87]:
reload(stats)

<module 'stats' from '/home/563/ab2313/Documents/PhD/modules/stats.py'>

In [92]:
%%time
futures = []
for arr in data_darray:
    future = client.submit(polyfit_2, arr)
    futures.append(future)

results = np.array(client.gather(futures))

CPU times: user 3.91 s, sys: 1.47 s, total: 5.38 s
Wall time: 5.22 s


In [93]:
results.shape

(500, 100000)

In [139]:
%%time
stats.trend_fit(data_xarray_stacked.chunk({'time':-1}), method='polynomial', order=2).compute()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 381.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 2.64 s, sys: 2.52 s, total: 5.16 s
Wall time: 9.86 s


<xarray.DataArray 'tas' (stacked_coord: 100000, time: 500)>
array([[0.47675731, 0.47695842, 0.47715899, ..., 0.51070741, 0.51064238,
        0.51057682],
       [0.50680421, 0.50699132, 0.50717761, ..., 0.49818185, 0.49795922,
        0.49773576],
       [0.525027  , 0.52494053, 0.52485411, ..., 0.48821201, 0.48815038,
        0.4880888 ],
       ...,
       [0.45206937, 0.45244934, 0.45282844, ..., 0.53438953, 0.53433997,
        0.53428954],
       [0.47929532, 0.4794988 , 0.47970146, ..., 0.47835133, 0.47814322,
        0.47793428],
       [0.48446703, 0.48465965, 0.48485149, ..., 0.48237302, 0.48217117,
        0.48196853]])
Coordinates:
  * time           (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2025-05-14
  * stacked_coord  (stacked_coord) object MultiIndex
  * latitude       (stacked_coord) float64 -90.0 -90.0 -90.0 ... 90.0 90.0 90.0
  * longitude      (stacked_coord) float64 -180.0 -176.4 -172.7 ... 176.4 180.0

In [ ]:
import statsmodels.api as sm 
lowess = sm.nonparametric.lowess

def apply_lowess(endog):
    exog = np.arange(endog.shape[0])
    return lowess(endog=endog, exog=exog, return_sorted=False)

In [126]:
data_darray[:100, :100].shape

(100, 100)

In [120]:
data_darray.shape

(500, 100000)

In [127]:
%%time
futures = []
for arr in data_darray[:100, :100]:
    future = client.submit(apply_lowess, arr)
    futures.append(future)

results = np.array(client.gather(futures))

CPU times: user 442 ms, sys: 36 ms, total: 478 ms
Wall time: 478 ms


In [140]:
data_xarray_stacked[:100, :100].shape

(100, 100)

In [143]:
data_xarray_stacked = data_xarray_stacked.persist()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 381.47 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [144]:
%%time
stats.trend_fit(data_xarray_stacked[:100, :100].chunk({'time':-1}), method='lowess').compute()

CPU times: user 933 ms, sys: 1 s, total: 1.94 s
Wall time: 2.61 s


<xarray.DataArray 'tas' (stacked_coord: 100, time: 100)>
array([[0.36035921, 0.36241968, 0.3641444 , ..., 0.59398762, 0.59728144,
        0.60035277],
       [0.43275223, 0.44372084, 0.4542743 , ..., 0.58344006, 0.5859405 ,
        0.58900029],
       [0.34940194, 0.36447908, 0.37921206, ..., 0.42042197, 0.40138314,
        0.38207588],
       ...,
       [0.55274161, 0.5604883 , 0.5678807 , ..., 0.51534985, 0.51057012,
        0.50551504],
       [0.52238669, 0.52680395, 0.53108738, ..., 0.50606048, 0.50751811,
        0.5088404 ],
       [0.35438869, 0.35155537, 0.34867809, ..., 0.31200528, 0.30048224,
        0.28926735]])
Coordinates:
  * time           (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2024-04-09
  * stacked_coord  (stacked_coord) object MultiIndex
  * latitude       (stacked_coord) float64 -90.0 -90.0 -90.0 ... -90.0 -90.0
  * longitude      (stacked_coord) float64 -180.0 -176.4 -172.7 ... 176.4 180.0

In [132]:
%%time
np.apply_along_axis(apply_lowess, 0, data_darray[:100, :100])

CPU times: user 1.02 s, sys: 131 ms, total: 1.16 s
Wall time: 994 ms


array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.30755851,  0.51767919,  0.45947733, ...,  0.61133142,
          0.56401163,  0.27457689]],

       [[ 1.        ,  1.        ,  1.        , ...,  1.        ,
          1.        ,  1.        ],
        [ 0.31494607,  0.51832842,  0.46266754, ...,  0.61058025,
          0.55847517,  0.28419375]],

       [[ 2.        ,  2.        ,  2.        , ...,  2.        ,
          2.        ,  2.        ],
        [ 0.32234437,  0.51891654,  0.46578922, ...,  0.6097914 ,
          0.55297186,  0.2938063 ]],

       ...,

       [[97.        , 97.        , 97.        , ..., 97.        ,
         97.        , 97.        ],
        [ 0.56086299,  0.62570672,  0.53388613, ...,  0.53276491,
          0.50535743,  0.36647397]],

       [[98.        , 98.        , 98.        , ..., 98.        ,
         98.        , 98.        ],
        [ 0.56118627,  0.62885175,  0.53226351, ...,  0.53292

In [130]:
results.shape

(100, 100, 2)

# Fast lowess function

In [25]:
import statsmodels.api as sm 
lowess = sm.nonparametric.lowess

def apply_lowess(endog):
    exog = np.arange(endog.shape[0])
    return lowess(endog=endog, exog=exog, return_sorted=False)

In [7]:
data_xarray = data_xarray.isel(time=slice(0, 50), latitude=slice(0, 50), longitude=slice(0, 50))

In [8]:
data_xarray

<xarray.DataArray 'tas' (time: 50, latitude: 50, longitude: 50)>
dask.array<getitem, shape=(50, 50, 50), dtype=float64, chunksize=(50, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2024-02-19
  * latitude   (latitude) float64 -90.0 -89.82 -89.64 ... -81.53 -81.35 -81.17
  * longitude  (longitude) float64 -180.0 -176.4 -172.7 ... -9.091 -5.455 -1.818

In [9]:
# data_xarray = data_xarray.compute()

In [10]:
dim ='time'

In [11]:
coords = np.array(list(data_xarray.coords))
coords

array(['time', 'latitude', 'longitude'], dtype='<U9')

In [12]:
coords_to_stack = coords[coords != dim]
coords_to_stack

array(['latitude', 'longitude'], dtype='<U9')

In [13]:
data_xarray_stacked = data_xarray.stack(stacked_coord=coords_to_stack.tolist())
data_xarray_stacked.shape

(50, 2500)

In [14]:
# # Delete me: just subsetting for testing purposes
# data_xarray_stacked = data_xarray_stacked.isel(time=slice(0, 40))
# data_xarray_stacked.shape

In [15]:
time_axis_num = data_xarray_stacked.get_axis_num('time')
time_axis_num

0

In [16]:
if time_axis_num != 0:
    data_xarray_stacked.transpose('time', ...)

In [17]:
time_axis_num = 0

In [18]:
data_dask_array = data_xarray_stacked.data

In [19]:
data_dask_array = data_dask_array.persist()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 95.37 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [20]:
data_dask_array.shape

(50, 2500)

In [21]:
data_dask_array = data_dask_array.rechunk((50, 250))

In [22]:
data_dask_array

dask.array<rechunk-merge, shape=(50, 2500), dtype=float64, chunksize=(50, 250), chunktype=numpy.ndarray>

In [26]:
%%time
futures = []
for arr in data_dask_array:
    future = client.submit(apply_lowess, arr)
    futures.append(future)

results = np.array(client.gather(futures))

- Receive client connection: Client-worker-9dcc2cac-df74-11ee-86c9-0000079cfe80
- Starting established connection to tcp://127.0.0.1:45840
- Receive client connection: Client-worker-9e869844-df74-11ee-86b4-0000079cfe80
- Starting established connection to tcp://127.0.0.1:45862
- Receive client connection: Client-worker-9e8768cb-df74-11ee-86c0-0000079cfe80
- Starting established connection to tcp://127.0.0.1:45876
- Receive client connection: Client-worker-9e86f77b-df74-11ee-86bd-0000079cfe80
- Starting established connection to tcp://127.0.0.1:45874
- Receive client connection: Client-worker-9e874500-df74-11ee-86b8-0000079cfe80
- Starting established connection to tcp://127.0.0.1:45886
- Receive client connection: Client-worker-9e88f5f7-df74-11ee-86c4-0000079cfe80
- Starting established connection to tcp://127.0.0.1:45900
- Receive client connection: Client-worker-9e8ad7fc-df74-11ee-86cc-0000079cfe80
- Starting established connection to tcp://127.0.0.1:45916
CPU times: user 3.71 s, sys

In [27]:
results.shape

(50, 2500)

In [32]:
%%time
stats.trend_fit(data_xarray, method='lowess').compute()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 95.37 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: user 1.85 s, sys: 523 ms, total: 2.37 s
Wall time: 10.7 s


<xarray.DataArray 'tas' (latitude: 50, longitude: 50, time: 50)>
array([[[0.39778883, 0.41531028, 0.43237789, ..., 0.35071992,
         0.34003614, 0.32953646],
        [0.50515645, 0.5120805 , 0.51859349, ..., 0.44727234,
         0.4466444 , 0.44619446],
        [0.50720963, 0.50608832, 0.50501912, ..., 0.39721816,
         0.38667838, 0.37583071],
        ...,
        [0.73087084, 0.71319439, 0.69565791, ..., 0.51530698,
         0.51748538, 0.52021662],
        [0.47430324, 0.47939609, 0.48456358, ..., 0.4332131 ,
         0.43355867, 0.43371142],
        [0.363779  , 0.3676202 , 0.37112152, ..., 0.60033318,
         0.60695431, 0.61383718]],

       [[0.4840642 , 0.49287228, 0.50060144, ..., 0.49200346,
         0.48677408, 0.48124422],
        [0.49799805, 0.4947158 , 0.49115715, ..., 0.57421283,
         0.58008412, 0.58517817],
        [0.57555883, 0.57087902, 0.56642239, ..., 0.48304933,
         0.47454201, 0.46607499],
...
        [0.44866143, 0.4593104 , 0.47020012, ..., 0.26742126,
         0.25065469, 0.23358755],
        [0.39870859, 0.40141421, 0.40463507, ..., 0.35984542,
         0.34484587, 0.33001035],
        [0.40854439, 0.41445827, 0.42067789, ..., 0.34878834,
         0.34071786, 0.33275045]],

       [[0.56169388, 0.5551386 , 0.5485168 , ..., 0.41006934,
         0.39267716, 0.3748986 ],
        [0.37682859, 0.39842013, 0.41903272, ..., 0.56170907,
         0.5719807 , 0.58243805],
        [0.50456039, 0.50481792, 0.50544458, ..., 0.49217256,
         0.500508  , 0.50913565],
        ...,
        [0.54315822, 0.53490163, 0.52700049, ..., 0.55067407,
         0.55805324, 0.56526537],
        [0.46901009, 0.47629449, 0.48290678, ..., 0.39924289,
         0.41039613, 0.4221231 ],
        [0.53048136, 0.52697658, 0.52312278, ..., 0.62363402,
         0.62900409, 0.63417145]]])
Coordinates:
  * time       (time) datetime64[ns] 2024-01-01 2024-01-02 ... 2024-02-19
  * latitude   (latitude) float64 -90.0 -89.82 -89.64 ... -81.53 -81.35 -81.17
  * longitude  (longitude) float64 -180.0 -176.4 -172.7 ... -9.091 -5.455 -1.818